# Библиотеки

In [49]:
import pandas as pd
import numpy as np
import neptune

from NN import simple_torchpl
from load_data import load
from pl_framework import nn_training
from cv import get_indices

import torch

In [50]:
neptune.init('iliaavilov/SIBUR')

Project(iliaavilov/SIBUR)

In [51]:
import os
os.environ['NEPTUNE_API_TOKEN']="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiYzI0ZTMzMDQtZTFmZi00ZjgxLWI4NGQtZGJiOWQyNDJiYjE5In0="
os.environ['NEPTUNE_PROJECT']="iliaavilov/SIBUR"
os.environ['NEPTUNE_NOTEBOOK_ID']="6eddcb21-b0fa-4073-a5c6-f26572800a8b"
os.environ['NEPTUNE_NOTEBOOK_PATH']="Desktop/Projects/sibur_2020/modeling.ipynb"

In [ ]:
exp = neptune.project.get_experiments('SIB-186')[0]
pd.read_csv(exp._backend.get_channel_points_csv(exp, channel_internal_id = '2bbf878c-15d5-43ca-9fa3-c4eed9ca64bf'), 
            header = None)

In [ ]:
channel_names = ['cv_scores']
channels_data = {}
channels_by_name = exp.get_channels()
for channel_name in channel_names:
    channel_id = channels_by_name[channel_name].id

    channels_data[channel_name] = pd.read_csv(
        exp._backend.get_channel_points_csv(exp, channel_id),
        header=None,
#            names=['x_{}'.format(channel_name), 'y_{}'.format(channel_name)],
        dtype=str
    )

In [ ]:
values = exp._backend.get_channel_points_csv(exp, channel_id).getvalue()
values.split('\n')

In [ ]:
neptune.project.get_experiments('SIB-186')[0].get_numeric_channels_values('cv_scores')

In [ ]:
neptune.project.get_experiments('SIB-186')[0]._get_channel('channel_name', 'text')

In [ ]:
neptune.project.get_experiments('SIB-186')[0].get_channels()

# Константы

In [52]:
data_path = 'data/'
random_state = 54321
pd.options.display.max_rows = 999
features = ['A_rate', 'A_CH4', 'A_C2H6', 'A_C3H8', 'A_iC4H10', 'A_nC4H10',
            'A_iC5H12', 'A_nC5H12', 'A_C6H14', 'B_rate']

# Загрузка данных

In [82]:
train_features, train_targets, test_features = load(data_path)

In [132]:
ff = pd.DataFrame({'tt': [0, 1, 2], 'pp': [10, 11, 12], 'uu': [30,40, 50]})

In [136]:
ff.mean(axis = 1)

0    13.333333
1    17.333333
2    21.333333
dtype: float64

In [83]:
cv = get_indices(train_targets, [(pd.to_datetime('2020-03-01 00:00:00'), pd.to_datetime('2020-03-15 00:00:00')),
                                 (pd.to_datetime('2020-03-15 00:00:00'), pd.to_datetime('2020-03-31 00:00:00')),
#                                 (pd.to_datetime('2020-03-31 00:00:00'), pd.to_datetime('2020-04-15 00:00:00')),
                                 (pd.to_datetime('2020-04-15 00:00:00'), pd.to_datetime('2020-04-30 00:00:00'))
                                ]
                )

In [87]:
ts = test_features['timestamp'].values
train_targets = train_targets.drop('timestamp', axis = 'columns')
test_features = test_features.drop('timestamp', axis = 'columns')
train_features = train_features.drop('timestamp', axis = 'columns')

# Предсказания для теста

In [ ]:
params_model = {'seq_len': 46, 'n_back': 57, 'n_h_1': 239, 'batch_size': 140, 'p_1': 0.0907928940794423, 'activation1': 'Tanhshrink', 'lr': 0.00021205814832404442, 'weight_decay': 0.021715440825812504, 'mean_best_iter': 626, 'n_in': 10, 'optimizer': 'AdamW', 'loss': torch.nn.L1Loss, 'n_out': 4}

In [ ]:
def make_seqs(n, data):
    final_X = []

    def seq2seq(x):
        seqs_X.append(np.array(x).reshape(-1, 1))
        return (5)

    for col in data.columns:
        seqs_X = []
        data[col].rolling(n).apply(seq2seq)
        seqs_X = np.array(seqs_X)
        final_X.append(seqs_X)

    final_X = np.concatenate(final_X, axis=2)
    return (final_X)


all_features = train_features.append(test_features, ignore_index = True)

seq_len = params_model['seq_len']
params_model.pop('seq_len')

batch_size = params_model['batch_size']
params_model.pop('batch_size')

mean_best_iter = params_model['mean_best_iter']
params_model.pop('mean_best_iter')

n_back = params_model['n_back']
params_model.pop('n_back')

all_features = all_features.shift(n_back)
all_features = all_features.dropna()

all_features = make_seqs(seq_len, all_features)
X_train = all_features[:(train_features.shape[0] - n_back-seq_len+1)]
X_test = all_features[(train_features.shape[0] - n_back-seq_len+1):]

X_train = X_train
X_test = X_test

y_train  = train_targets
y_train = y_train[(seq_len + n_back-1):].reset_index(drop = True)

In [ ]:
X_test.shape

In [ ]:
test_features

In [ ]:
t = 0
for fold in cv:
    fold[0] = fold[0][(seq_len + n_back - 1):]
    for idx1 in range(0, len(fold[0])):
        fold[0][idx1] = fold[0][idx1] - seq_len - n_back + 1

    for idx2 in range(0, len(fold[1])):
        fold[1][idx2] = fold[1][idx2] - seq_len - n_back + 1
    cv[t] = fold
    t = t + 1

In [ ]:
cv_new = [cv[0]]

In [ ]:
my_boiii = nn_training(simple_torchpl, X_train, y_train)

my_boiii.train(min_epochs=mean_best_iter,
               max_epochs=mean_best_iter,
               model_params=params_model,
               batch_size=batch_size,
               fold=cv_new[0],
               val_fold=False)

In [ ]:
my_model1 = my_boiii.trained_model
my_model1.eval()

In [ ]:
predictions = my_model1(torch.from_numpy(X_test).float()).detach().numpy()
submission = pd.DataFrame(predictions, columns = ['B_C2H6', 'B_C3H8', 'B_iC4H10', 'B_nC4H10'])
submission['timestamp'] = ts
submission.to_csv('submission.csv', index = False)

In [ ]:
#neptune.project.get_experiments('SIB-71')[0].log_metric('leaderboard_mape', 7.4781)